## Ejemplo 4: Agrupando datos con `groupby`

### 1. Objetivos:
    - Aprender a usar `group-by` para segmentar nuestros conjuntos de datos y aplicar funciones agregadoras a cada segmento.
 
---
    
### 2. Desarrollo:

#### a) Segmentando datos con `groupby`

En nuestro Reto pasado construimos un nuevo conjunto de datos agregando la información de las tablas `occupations` y `age_ranges` a la tabla `users`, pero por si aún no lo has hecho podemos usar el archivo `users-full.csv` en la carpeta Datasets:

In [2]:
import pandas as pd

In [3]:
users = pd.read_csv("../Ejemplo-03/users_full.csv", index_col=0)

users.head()

,gender,age,occupation_id,cp,occupation
0,F,1,10,48067,K-12 student
1,F,1,10,48067,K-12 student
2,F,1,10,48067,K-12 student
3,M,1,10,48073,K-12 student
4,M,1,10,48073,K-12 student


Vamos a ver qué pasa si agrupamos nuestro conjunto usando la columna `gender` y la función `groupby()`:

`dataframe.groupby(-columna-)`

In [4]:
users.groupby("gender")

Cuando se hace una agrupación y tener un resultado más tangible tenemos que aplicar funciones agregadoras a nuestro objeto `groupby`, por ejemplo podemos usar:

`dataframe.groupby(-columna-).size()`

In [5]:
users.groupby("gender").size()

gender
F    15381
M    38979
dtype: int64

`size` nos hace un conteo de cuántas muestras hay en cada grupo y regresa el total. Ahora podemos ver entonces que hay 1709 mujeres y 4331 hombres en nuestro dataset.

También podemos pedir columnas específicas de nuestros grupos y aplicar agregaciones a cada columna de la forma:

`dataframe.groupby(-columna-)[-columna segunda clasificación-].value_counts()`

entonces ahora veamos por género la cantidad de personas por categoría de ocupaciones:

In [6]:
users.groupby("gender")["occupation"].value_counts()

gender  occupation             
F       college/grad student       2106
        academic/educator          1881
        other or not specified     1392
        executive/managerial       1251
        doctor/health care          918
        clerical/admin              900
        artist                      819
        homemaker                   801
        sales/marketing             711
        writer                      702
        other or, not specified     696
        K-12 student                594
        technician/engineer         468
        self-employed               459
        programmer                  450
        retired                     306
        customer service            279
        scientist                   252
        lawyer                      198
        unemployed                  135
        tradesman/craftsman          36
        farmer                       27
M       executive/managerial       4860
        college/grad student       4725
        

Podemos usar dos o más columnas para agrupar también. Lo que sucede es que el dataset se agrupa usando la primer columna, y luego, dentro de cada grupo se hace una segunda agrupación usando la segunda columna, por ejemplo:

`dataframe.groupby([-lista de columnas])[-columna-].value_counts()`

por ejemplo por `gender` y `age_range`:

In [11]:
users.groupby( ["gender", "age"] )["occupation"].value_counts().head(20)

gender  age  occupation             
F       1    K-12 student                567
             other or not specified       54
             other or, not specified      27
             artist                       18
             unemployed                   18
             academic/educator             9
             executive/managerial          9
        18   college/grad student       1467
             other or not specified      192
             academic/educator           162
             sales/marketing             135
             writer                      126
             other or, not specified      96
             artist                       81
             clerical/admin               81
             technician/engineer          54
             unemployed                   54
             customer service             45
             homemaker                    45
             K-12 student                 27
Name: occupation, dtype: int64

Y que pasa si queremos conocer la cantidad de usuarios por cada rango de edad para cada género podemos usar la forma:

`df.groupby(-lista de columnas-).size()`

In [12]:
users.groupby( ["gender", "age"] ).size()

gender  age
F       1        702
        18      2682
        25      5022
        35      3042
        45      1701
        50      1314
        56       918
M       1       1296
        18      7245
        25     13842
        35      7695
        45      3249
        50      3150
        56      2502
dtype: int64

Aquí hemos segmentado nuestro dataset en dos niveles. En el primer nivel, podemos obtener datasets independientes para cada género, primero guardemos el resultado anterior en otra variable:

In [13]:
users_counts = users.groupby( ["gender", "age"] ).size()

users_counts

gender  age
F       1        702
        18      2682
        25      5022
        35      3042
        45      1701
        50      1314
        56       918
M       1       1296
        18      7245
        25     13842
        35      7695
        45      3249
        50      3150
        56      2502
dtype: int64

Ahora consultamos el conteo sólo para el género femenino con `df.loc["F"]`, ya que así accedemos al primer nivel:

In [14]:
users_counts.loc["F"]

age
1      702
18    2682
25    5022
35    3042
45    1701
50    1314
56     918
dtype: int64

En un segundo nivel, podemos obtener datasets por cada rango de edades en cada género, por ejemplo cuantos usuarios hay con género femenino y en el rango de 18 a 24 usando `df.loc[-indice 1er nivel-, -índice 2o nivel-]`

In [15]:
users_counts.loc["F", 18]

2682

¡Genial!

Ahora, no todas las funciones están disponibles para ser aplicadas a objetos `groupby`, hay algunas funciones que no podemos utilizar directamente y que para poder aplicarlas necesitamos usar el método `agg` (aggregations). `agg` recibe una función o una lista de funciones y se las aplica a las columnas solicitadas de cada grupo.

Por ejemplo, podemos encontrar la "moda" (la categoría que más veces aparece en una columna específica) usando la forma:

`df.groupby(-columna-)[-columna-].agg( pd.Series.mode )`

en este caso queremos encontrar la ocupación que más realizan los usuarios por género:

In [16]:
users.groupby("gender")["occupation"].agg( pd.Series.mode )

gender
F    college/grad student
M    executive/managerial
Name: occupation, dtype: object

Podemos aplicar la función a dos columnas al mismo tiempo usando la forma:

`df.groupby(-columna-)[-lista de columnas-].agg( pd.Series.mode )`

en este caso queremos encontrar la ocupación que más realizan los usuarios y el rángo de edad donde hay más usuarios agrupado por género:

In [18]:
users.groupby("gender")[ ["occupation", "age"] ].agg( pd.Series.mode )

,occupation,age
gender,,
F,college/grad student,25
M,executive/managerial,25


Y también podemos aplicar varias funciones al mismo tiempo pasándole a `agg` una lista de funciones. En este caso vamos a usar algunos análisis estadísticos a la columna `age_id`. En realidad estos análisis no van a ser precisos porque esta columna contiene ids que representan rangos de edades, no edades como tal. Pero considéralo un simple ejemplo para ver cómo funcionan las herramientas:

In [20]:
users.groupby('gender')['age'].agg(['mean', 'median', 'std'])

,mean,median,std
gender,,,
F,30.859567,25,13.239120
M,30.552297,25,12.755801


Ahora si, los retos son todos tuyos!